In [5]:
import pandas as pd

def remove_duplicates_and_save(input_csv_path, output_csv_path):
    """
    CSV 파일을 읽어 'food_name'과 'food_id'를 기준으로 중복을 제거하고,
    결과를 새로운 CSV 파일로 저장합니다.

    Args:
        input_csv_path (str): 원본 CSV 파일 경로.
        output_csv_path (str): 중복 제거 후 저장할 CSV 파일 경로.
    """
    print(f"'{input_csv_path}' 파일 읽기 시작...")
    df = pd.read_csv(input_csv_path, encoding='utf-8')
    print(f"원본 데이터 크기: {len(df)} 행")

    # 'food_name'과 'food_id' 기준으로 중복 제거
    # 'food_name'이 동일해도 'food_id'가 다를 수 있으므로 두 컬럼 모두를 기준으로 합니다.
    # keep='first'는 중복된 경우 첫 번째 항목을 유지합니다.
    df_cleaned = df.drop_duplicates(subset=['food_name'], keep='first')
    df_cleaned = df_cleaned.drop_duplicates(subset=['food_id'], keep='first')
    print(f"중복 제거 후 데이터 크기: {len(df_cleaned)} 행")

    # 새로운 CSV 파일로 저장
    df_cleaned.to_csv(output_csv_path, index=False, encoding='utf-8')
    print(f"중복 제거된 데이터를 '{output_csv_path}'에 저장 완료.")

# 사용 예시 (아래 경로를 실제 파일 경로로 수정하세요)
input_file = r"C:\Users\lkw45\PycharmProjects\food_scheduler\data\foods\combine_data.csv"
output_file = r"C:\Users\lkw45\PycharmProjects\food_scheduler\data\foods\combine_data_cleaned.csv"

remove_duplicates_and_save(input_file, output_file)


'C:\Users\lkw45\PycharmProjects\food_scheduler\data\foods\combine_data.csv' 파일 읽기 시작...


C:\Users\lkw45\AppData\Local\Temp\ipykernel_16500\1157545094.py:13: DtypeWarning: Columns (12,16,19,20,21,22,23,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_csv_path, encoding='utf-8')


원본 데이터 크기: 166248 행
중복 제거 후 데이터 크기: 148032 행
중복 제거된 데이터를 'C:\Users\lkw45\PycharmProjects\food_scheduler\data\foods\combine_data_cleaned.csv'에 저장 완료.


## 파일이 너무 크면 pandas로 읽어 지지 않음
## 모두 csv로 바꾼 후 laod
## 컬럼 많은 파일 합치기

## 하지만 카테고리 라벨이 없어서 로컬 LLM (Gemma3)를 사용하여 카테고리를 라벨링 하기로 결정!!

### DB 해야할 거
1. 음식 테이블의 스키마 정하기
2. 카테고리 LLM으로 채워넣기 (not null)
3. table 관계 짜서 mysql에 넣기
4. 입출력 코드 일반화
5. 서버와 연결

### 사용할 컬럼만 추려서 저장해둔 all_column.csv로 컬럼 가져오기

In [1]:
import pandas as pd
import os

In [2]:
all_columns_df = pd.read_csv("all_column.csv", encoding="utf-8-sig")

In [3]:
my_columns = all_columns_df.csv_name.dropna().to_list()

In [4]:
all_df = pd.DataFrame(columns=my_columns)
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 57 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   식품코드          0 non-null      object
 1   식품명           0 non-null      object
 2   데이터구분코드       0 non-null      object
 3   데이터구분명        0 non-null      object
 4   식품기원코드        0 non-null      object
 5   식품기원명         0 non-null      object
 6   식품대분류코드       0 non-null      object
 7   식품대분류명        0 non-null      object
 8   대표식품코드        0 non-null      object
 9   대표식품명         0 non-null      object
 10  식품중분류코드       0 non-null      object
 11  식품중분류명        0 non-null      object
 12  식품소분류코드       0 non-null      object
 13  식품소분류명        0 non-null      object
 14  식품세분류코드       0 non-null      object
 15  식품세분류명        0 non-null      object
 16  출처코드          0 non-null      object
 17  출처명           0 non-null      object
 18  품목제조보고번호      0 non-null      object
 19  업체명           0 non-

### 원본 데이터들에서 원하는 컬럼만 가져와 병합

In [23]:
file_dir = "source"

file_list = [table_file for table_file in os.listdir(file_dir) if table_file.endswith(".csv")]

In [24]:
for file in file_list:
    for chunk in pd.read_csv(str(os.path.join(file_dir, file)), chunksize=5000, encoding="utf-8-sig", dtype='object'):
      all_df = pd.concat([all_df, chunk.reindex(columns=my_columns)], ignore_index=True)

In [25]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613309 entries, 0 to 613308
Data columns (total 59 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   식품코드          613309 non-null  object
 1   식품명           613309 non-null  object
 2   데이터구분코드       613309 non-null  object
 3   데이터구분명        613309 non-null  object
 4   식품기원코드        397071 non-null  object
 5   식품기원명         397071 non-null  object
 6   식품대분류코드       397071 non-null  object
 7   식품대분류명        397071 non-null  object
 8   대표식품코드        397071 non-null  object
 9   대표식품명         397071 non-null  object
 10  식품중분류코드       397071 non-null  object
 11  식품중분류명        397071 non-null  object
 12  식품소분류코드       397071 non-null  object
 13  식품소분류명        397071 non-null  object
 14  식품세분류코드       397071 non-null  object
 15  식품세분류명        397071 non-null  object
 16  출처코드          613309 non-null  object
 17  출처명           613309 non-null  object
 18  품목제조보고번호      494246 non

### 식품 코드 중복 제거

In [26]:
unique_fid_all_df = all_df.groupby('식품코드').first()

In [54]:
unique_fid_all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166248 entries, D101-004160000-0001 to R320-025030002-0000
Data columns (total 58 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   식품명           166248 non-null  object
 1   데이터구분코드       166248 non-null  object
 2   데이터구분명        166248 non-null  object
 3   식품기원코드        166248 non-null  object
 4   식품기원명         166248 non-null  object
 5   식품대분류코드       166248 non-null  object
 6   식품대분류명        166248 non-null  object
 7   대표식품코드        166248 non-null  object
 8   대표식품명         166248 non-null  object
 9   식품중분류코드       166248 non-null  object
 10  식품중분류명        166248 non-null  object
 11  식품소분류코드       166248 non-null  object
 12  식품소분류명        166248 non-null  object
 13  식품세분류코드       166248 non-null  object
 14  식품세분류명        166248 non-null  object
 15  출처코드          166248 non-null  object
 16  출처명           166248 non-null  object
 17  품목제조보고번호      139968 non-null  object
 18

In [55]:
unique_fid_all_df.to_csv("combine_data_kr.csv", encoding="utf-8-sig")

### 다시 읽어서 컬럼을 한국어로 저장

In [8]:
unique_fid_all_df = all_df
for chunk in pd.read_csv("combine_data_kr.csv", chunksize=5000, encoding="utf-8-sig", dtype='object'):
      unique_fid_all_df = pd.concat([unique_fid_all_df, chunk])

In [9]:
korean_to_english_columns = {
    # FoodInfo 테이블
    '식품코드': 'food_id',
    '식품명': 'food_name',
    '데이터구분코드': 'data_type_code',
    '데이터구분명': 'data_type_name',
    '품목제조보고번호': 'mfg_report_no',
    '수입여부': 'is_imported',
    '데이터기준일자': 'reference_date',
    '폐기율(%)': 'disposal_rate_pct',

    # FoodCategory 테이블
    '식품대분류코드': 'major_category_code',
    '식품대분류명': 'major_category_name',
    '대표식품코드': 'representative_food_code',
    '대표식품명': 'representative_food_name',
    '식품중분류코드': 'medium_category_code',
    '식품중분류명': 'medium_category_name',
    '식품소분류코드': 'minor_category_code',
    '식품소분류명': 'minor_category_name',
    '식품세분류코드': 'detail_category_code',
    '식품세분류명': 'detail_category_name',

    # FoodSourceInfo 테이블
    '식품기원코드': 'origin_code',
    '식품기원명': 'origin_name',
    '출처코드': 'source_code',
    '출처명': 'source_name',
    '데이터생성방법코드': 'generation_method_code',
    '데이터생성방법명': 'generation_method_name',

    # FoodCompany 테이블
    '업체명': 'company_name',
    '제조사명': 'manufacturer_name',
    '원산지국코드': 'origin_country_code',
    '원산지국명': 'origin_country_name',
    '수입업체명': 'importer_name',
    '유통업체명': 'distributor_name',

    # FoodNutrition 테이블
    '식품중량': 'weight',
    '1회 섭취참고량': 'serving_size_g',
    '영양성분함량기준량': 'nutrient_reference_amount_g',
    '에너지(kcal)': 'energy_kcal',
    '수분(g)': 'moisture_g',
    '단백질(g)': 'protein_g',
    '지방(g)': 'fat_g',
    '회분(g)': 'ash_g',
    '탄수화물(g)': 'carbohydrate_g',
    '당류(g)': 'sugars_g',
    '식이섬유(g)': 'dietary_fiber_g',
    '칼슘(mg)': 'calcium_mg',
    '철(mg)': 'iron_mg',
    '인(mg)': 'phosphorus_mg',
    '칼륨(mg)': 'potassium_mg',
    '나트륨(mg)': 'sodium_mg',
    '비타민A(μg RAE)': 'vitamin_a_ug_rae',
    '레티놀(μg)': 'retinol_ug',
    '베타카로틴(μg)': 'beta_carotene_ug',
    '티아민(mg)': 'thiamin_mg',
    '리보플라빈(mg)': 'riboflavin_mg',
    '니아신(mg)': 'niacin_mg',
    '비타민 C(mg)': 'vitamin_c_mg',
    '비타민 D(μg)': 'vitamin_d_ug',
    '콜레스테롤(mg)': 'cholesterol_mg',
    '포화지방산(g)': 'saturated_fat_g',
    '트랜스지방산(g)': 'trans_fat_g'
}

In [12]:
unique_fid_all_df.rename(columns=korean_to_english_columns, inplace=True)

In [13]:
unique_fid_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166248 entries, 0 to 166247
Data columns (total 57 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   food_id                      166248 non-null  object
 1   food_name                    166248 non-null  object
 2   data_type_code               166248 non-null  object
 3   data_type_name               166248 non-null  object
 4   origin_code                  166248 non-null  object
 5   origin_name                  166248 non-null  object
 6   major_category_code          166248 non-null  object
 7   major_category_name          166248 non-null  object
 8   representative_food_code     166248 non-null  object
 9   representative_food_name     166248 non-null  object
 10  medium_category_code         166248 non-null  object
 11  medium_category_name         166248 non-null  object
 12  minor_category_code          166248 non-null  object
 13  minor_category

In [16]:
unique_fid_all_df.replace("해당없음", pd.NA, inplace=True)

In [21]:
unique_fid_all_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166248 entries, 0 to 166247
Data columns (total 57 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   food_id                      166248 non-null  object
 1   food_name                    166248 non-null  object
 2   data_type_code               166248 non-null  object
 3   data_type_name               166248 non-null  object
 4   origin_code                  166248 non-null  object
 5   origin_name                  166248 non-null  object
 6   major_category_code          166248 non-null  object
 7   major_category_name          166248 non-null  object
 8   representative_food_code     166248 non-null  object
 9   representative_food_name     166248 non-null  object
 10  medium_category_code         166248 non-null  object
 11  medium_category_name         102304 non-null  object
 12  minor_category_code          166248 non-null  object
 13  minor_category

In [23]:
unique_fid_all_df.to_csv("combine_data.csv", encoding="utf-8")